DIAG Server

## Host information (Query 11) (Host Info)

In [ ]:
-- Host information (Query 11) (Host Info)
SELECT host_platform, host_distribution, host_release, 
       host_service_pack_level, host_sku, os_language_version 
FROM sys.dm_os_host_info WITH (NOLOCK) OPTION (RECOMPILE); 

[Hardware and Software Requirements for Installing SQL Server](https://bit.ly/2y3ka5L)

## SQL Server NUMA Node information (Query 12) (SQL Server NUMA Info)

In [ ]:
-- SQL Server NUMA Node information  (Query 12) (SQL Server NUMA Info)
SELECT node_id, node_state_desc, memory_node_id, processor_group, cpu_count, online_scheduler_count, 
       idle_scheduler_count, active_worker_count, avg_load_balance, resource_monitor_state
FROM sys.dm_os_nodes WITH (NOLOCK) 
WHERE node_state_desc <> N'ONLINE DAC' OPTION (RECOMPILE);

Gives you some useful information about the composition and relative load on your NUMA nodes

- You want to see an equal number of schedulers on each NUMA node
- Watch out if SQL Server 2019 Standard Edition has been installed on a physical or virtual machine with more than four sockets or more than 24 physical cores

[sys.dm\_os\_nodes (Transact-SQL)](https://bit.ly/2pn5Mw8)

[How to Balance SQL Server Core Licenses Across NUMA Nodes](https://bit.ly/3i4TyVR)

## Good basic information about OS memory amounts and state (Query 13) (System Memory)

In [ ]:
-- Good basic information about OS memory amounts and state  (Query 13) (System Memory)
SELECT total_physical_memory_kb/1024 AS [Physical Memory (MB)], 
       available_physical_memory_kb/1024 AS [Available Memory (MB)], 
       total_page_file_kb/1024 AS [Page File Commit Limit (MB)],
	   total_page_file_kb/1024 - total_physical_memory_kb/1024 AS [Physical Page File Size (MB)],
	   available_page_file_kb/1024 AS [Available Page File (MB)], 
	   system_cache_kb/1024 AS [System Cache (MB)],
       system_memory_state_desc AS [System Memory State]
FROM sys.dm_os_sys_memory WITH (NOLOCK) OPTION (RECOMPILE);

You want to see "Available physical memory is high" for System Memory State. This indicates that you are not under external memory pressure

Possible System Memory State values:

- Available physical memory is high
- Physical memory usage is steady
- Available physical memory is low
- Available physical memory is running low
- Physical memory state is transitioning

[sys.dm\_os\_sys\_memory (Transact-SQL)](https://bit.ly/2pcV0xq)

## Get information about your cluster nodes and their status (Query 14) (Cluster Node Properties)

> Skip this query if you know you don't have a clustered instance.

In [ ]:
-- Get information about your cluster nodes and their status  (Query 14) (Cluster Node Properties)
-- (if your database server is in a failover cluster)
SELECT NodeName, status_description, is_current_owner
FROM sys.dm_os_cluster_nodes WITH (NOLOCK) OPTION (RECOMPILE);

- Knowing which node owns the cluster resources is critical
- Especially when you are installing Windows or SQL Server updates
- You will see no results if your instance is not clustered

## Hardware information from SQL Server 2019 (Query 17) (Hardware Info)

In [ ]:
-- Hardware information from SQL Server 2019  (Query 17) (Hardware Info)
SELECT cpu_count AS [Logical CPU Count], scheduler_count, 
       (socket_count * cores_per_socket) AS [Physical Core Count], 
       socket_count AS [Socket Count], cores_per_socket, numa_node_count,
       physical_memory_kb/1024 AS [Physical Memory (MB)], 
       max_workers_count AS [Max Workers Count], 
	   affinity_type_desc AS [Affinity Type], 
       sqlserver_start_time AS [SQL Server Start Time],
	   DATEDIFF(hour, sqlserver_start_time, GETDATE()) AS [SQL Server Up Time (hrs)],
	   virtual_machine_type_desc AS [Virtual Machine Type], 
       softnuma_configuration_desc AS [Soft NUMA Configuration], 
	   sql_memory_model_desc, 
	   container_type_desc -- New in SQL Server 2019
FROM sys.dm_os_sys_info WITH (NOLOCK) OPTION (RECOMPILE);

Gives you some good basic hardware information about your database server

**Note:** virtual\_machine\_type\_desc of HYPERVISOR does not automatically mean you are running SQL Server inside of a VM. It merely indicates that you have a hypervisor running on your host

[sys.dm\_os\_sys\_info (Transact-SQL)](https://bit.ly/2pczOYs)

Soft NUMA configuration was a new column for SQL Server 2016

- OFF = Soft-NUMA feature is OFF
- ON = SQL Server automatically determines the NUMA node sizes for Soft-NUMA
- MANUAL = Manually configured soft-NUMA

[Configure SQL Server to Use Soft-NUMA (SQL Server)](https://bit.ly/2HTpKJt)

sql\_memory\_model\_desc values (Added in SQL Server 2016 SP1)

- CONVENTIONAL
- LOCK\_PAGES
- LARGE\_PAGES

## Get System Manufacturer and model number from SQL Server Error log (Query 18) (System Manufacturer)

In [ ]:
-- Get System Manufacturer and model number from SQL Server Error log (Query 18) (System Manufacturer)
EXEC sys.xp_readerrorlog 0, 1, N'Manufacturer';

This can help you determine the capabilities and capacities of your database server

- Can also be used to confirm if you are running in a VM
- This query might take a few seconds if you have not recycled your error log recently
- This query will return no results if your error log has been recycled since the instance was started

## Get BIOS date from Windows Registry (Query 19) (BIOS Date)

In [ ]:
-- Get BIOS date from Windows Registry (Query 19) (BIOS Date)
EXEC sys.xp_instance_regread N'HKEY_LOCAL_MACHINE', N'HARDWARE\DESCRIPTION\System\BIOS', N'BiosReleaseDate';

Helps you understand whether the main system BIOS is up to date, and the possible age of the hardware

- Not as useful for virtualization
- Does not work on Linux

## Get processor description from Windows Registry (Query 20) (Processor Description)

In [ ]:
-- Get processor description from Windows Registry  (Query 20) (Processor Description)
EXEC sys.xp_instance_regread N'HKEY_LOCAL_MACHINE', N'HARDWARE\DESCRIPTION\System\CentralProcessor\0', N'ProcessorNameString';

Gives you the model number and rated clock speed of your processor(s)

- Your processors may be running at less than the rated clock speed due to the Windows Power Plan or hardware power management
- Does not work on Linux

You can use [CPU-Z](https://bit.ly/QhR6xF) to get your actual CPU core speed and a lot of other useful information

You can learn more about processor selection for SQL Server by [following this link](https://bit.ly/2F3aVlP)